In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pandas_datareader as web
import datetime as dt

In [2]:
# pip install pandas_datareader

In [3]:
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM

## Loading Data

In [4]:
company = 'AMZN'

In [5]:
start = dt.datetime(2012,1,1)
end = dt.datetime(2020,1,1)

In [6]:
data = web.DataReader(company, 'yahoo', start, end)

## Preparing Data

In [7]:
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data['Close'].values.reshape(-1, 1))

In [8]:
prediction_days = 60

In [9]:
x_train = []
y_train = []

for i in range(prediction_days, len(scaled_data)):
    x_train.append(scaled_data[i-prediction_days:i, 0])
    y_train.append(scaled_data[i, 0])


In [10]:
x_train, y_train = np.array(x_train), np.array(y_train)
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

## Building the model

In [11]:
model = Sequential()

In [12]:
model.add(LSTM(units=50, return_sequences=True,input_shape=(x_train.shape[1], 1)))
model.add(Dropout(0.2))
model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=50))
model.add(Dropout(0.2))
model.add(Dense(units=1))


In [ ]:
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(x_train, y_train, epochs=35, batch_size=32)

## Load Test Data

In [ ]:
test_start = dt.datetime(2020,1,1)
test_end = dt.datetime.now()

In [ ]:
test_data = web.DataReader(company, 'yahoo', test_start, test_end)
actual_prices = test_data['Close'].values

In [ ]:
total_dataset = pd.concat((data['Close'], test_data['Close']), axis=0)

In [ ]:
total_dataset

In [ ]:
model_inputs = total_dataset[len(total_dataset) - len(test_data) - prediction_days:].values

In [ ]:
model_inputs = model_inputs.reshape(-1, 1)
model_inputs = scaler.transform(model_inputs)

## Make Predictions on Test Data

In [ ]:
x_test = []

for i in range(prediction_days, len(model_inputs)+1):
    x_test.append(model_inputs[i-prediction_days:i, 0])

x_test = np.array(x_test)
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

In [ ]:
predicted_prices = model.predict(x_test)
predicted_prices = scaler.inverse_transform(predicted_prices)

## Plotting the test Predictions

In [ ]:
plt.plot(actual_prices, color='blue', label=f'Actual {company} Price')
plt.plot(predicted_prices, color='red', label=f'Predicted {company} Price')
plt.title(f'{company} Share Price')
plt.xlabel('Time')
plt.ylabel(f'{company} Share Price')
plt.legend()
plt.show()


## Predicting the Next Day

In [ ]:
real_data = [model_inputs[len(model_inputs)+1 - prediction_days: len(model_inputs + 1), 0]]

In [ ]:
real_data = np.array(real_data)

In [ ]:
real_data = np.reshape(real_data, (real_data.shape[0], real_data.shape[1], 1))

In [ ]:
prediction = model.predict(real_data)
prediction = scaler.inverse_transform(prediction)
print(f'Prediction: {prediction}')